In [ ]:
import sys

sys.path.append("/Users/evanracah/Dropbox/projects/rl-representation-learning/")

In [2]:
import wandb
import pandas as pd
import numpy as np

In [3]:
from src.atari_ram_annotations import summary_key_dict, atari_dict, unused_keys

In [4]:
def remove_dicts_in_dict(dic):
    rems = []
    for k,v in dic.items():
        if isinstance(v,dict) or isinstance(v,list):
            rems.append(k)
    for k in rems:            
        if isinstance(dic[k], list):
            for i,v in enumerate(dic[k]):
                dic[k+"_"+str(i)] = v
        dic.pop(k)
        
    return dic
            

In [5]:
def get_dicts_for(method):
    rem_games = []
    api = wandb.Api()
    if method in ["random_cnn","supervised"]:
        runs = list(api.runs("curl-atari/curl-atari-2", {"config.method": method,
                                                 "config.collect_mode":"random_agent",
                                                 "state": "finished"}))
    else:
        runs = list(api.runs("curl-atari/curl-atari-2", {"config.method": method,
                                                         "config.collect_mode":"random_agent",
                                                         "config.train_encoder":True,
                                                       "config.feature_size":256,
                                                         "state": "finished", 
                                                        #"config.patience":10
                                                        }))



    runs_info = {run.config["env_name"].replace("NoFrameskip-v4","") + "_" + run.name :remove_dicts_in_dict(run.config) for run in runs}
    summary_metrics = {run.config["env_name"].replace("NoFrameskip-v4","") + "_" + run.name :run.summary_metrics for run in runs }
    summary_metrics = {k:v for k,v in summary_metrics.items() if k.split("_")[0] not in rem_games}
    run_summary_dict = {name:{} for name in summary_metrics.keys()}
    for run_name, run in summary_metrics.items():
        for summary_name, key_list in summary_key_dict.items():
            run_mean_values = {k.lower():v for k,v in run.items() if "acc" in k and "stderr" not in k  and  any(sum_key.lower() in k.lower() for sum_key in key_list)}
            #print(run_mean_values)
            #run_stderr_values = {k:v for k,v in run.items() if "stderr" in k and "mean" not in k and any(sum_key in k.lower() for sum_key in key_list)}
            if len(run_mean_values) > 0:
                run_summary = np.mean(list(run_mean_values.values()))
                #overall_summary_dict[summary_name].append(run_summary)
                run_summary_dict[run_name][summary_name] = run_summary
                #print("\t",summary_name,run_summary_dict[run_name][summary_name])
        run_summary_dict[run_name]["overall"] = np.mean(list(run_summary_dict[run_name].values()))
                
    for game_name in atari_dict.keys():
        duplicates = [run_name for run_name in summary_metrics.keys() if game_name in run_name.lower()]
        num_duplicates = len(duplicates)
        if num_duplicates > 1:
            
            duplicates_info = [runs_info[run_name] for run_name in duplicates]
            print("Game {} is in summary {} times as:".format(game_name, num_duplicates))
            print("\t",duplicates)
            print("\tDifferences are: ")
            for info in duplicates_info[1:]:
                print("\t\t",set(duplicates_info[0].items()).symmetric_difference(set(info.items())))
            duplicates_time_stamps = [summary_metrics[run_name]["_timestamp"] for run_name in duplicates ]
            most_recent_index = np.argmax(duplicates_time_stamps)
            for i, run_name in enumerate(duplicates):
                if i != most_recent_index:
                    run_summary_dict.pop(run_name)
            
                

    

    return run_summary_dict, runs_info, runs 
        

In [6]:
def compute_overall_summary_dict(run_summary_dict):
    all_dups = []
    for game_name in atari_dict.keys():       
        duplicates = [run_name for run_name in run_summary_dict.keys() if game_name in run_name.lower()]
        all_dups.append(duplicates)
        num_duplicates = len(duplicates)
        if num_duplicates > 1:
            print("You still have duplicates!".format(game_name, num_duplicates))
            print("\t",duplicates)
            print("")
            
    overall_summary_dict = {k:[] for k in list(summary_key_dict.keys()) + ["overall"]}
    for run_name, key_dict in run_summary_dict.items():
        for key,value in key_dict.items():
            overall_summary_dict[key].append(value)
        
    overall_summary_dict = {k:np.mean(v) for k,v in overall_summary_dict.items()}


    return overall_summary_dict

In [7]:
run_summary_dict,runs_info, runs = get_dicts_for("bert")

Game boxing is in summary 2 times as:
	 ['Boxing_zyxbjp4e', 'Boxing_y4lugrs2']
	Differences are: 
		 {('sequence_length', 10), ('linear', True), ('steps_step', 4), ('sequence_length', 100), ('steps_step', 1), ('steps_end', 99), ('steps_end', 9)}
Game pitfall is in summary 2 times as:
	 ['Pitfall_i6r8odxn', 'Pitfall_2lwcgebo']
	Differences are: 
		 {('sequence_length', 10), ('linear', True), ('steps_step', 4), ('sequence_length', 100), ('steps_step', 1), ('steps_end', 99), ('steps_end', 9)}
Game pong is in summary 2 times as:
	 ['Pong_5xwqtld8', 'Pong_fmc1nnj8']
	Differences are: 
		 {('sequence_length', 10), ('linear', True), ('steps_step', 4), ('sequence_length', 100), ('steps_step', 1), ('steps_end', 99), ('steps_end', 9)}


/Users/evanracah/miniconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/evanracah/miniconda3/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [18]:
{run.config["env_name"]: run.summary_metrics["train_accuracy"] for run in runs}

{'PitfallNoFrameskip-v4': 0.4999872148036957,
 'PongNoFrameskip-v4': 0.5,
 'BoxingNoFrameskip-v4': 0.5,
 'MontezumaRevengeNoFrameskip-v4': 0.9148263335227966,
 'PrivateEyeNoFrameskip-v4': 0.9920879006385804,
 'AsteroidsNoFrameskip-v4': 0.828688383102417,
 'BerzerkNoFrameskip-v4': 0.8960811495780945,
 'BreakoutNoFrameskip-v4': 0.9630773067474364,
 'DefenderNoFrameskip-v4': 0.7527104616165161,
 'RiverraidNoFrameskip-v4': 0.945261299610138,
 'MsPacmanNoFrameskip-v4': 0.9263008236885072}

In [16]:
{run.config["env_name"]: run.summary_metrics["val_accuracy"] for run in runs}

{'PitfallNoFrameskip-v4': 0.4999999701976776,
 'PongNoFrameskip-v4': 0.5,
 'BoxingNoFrameskip-v4': 0.5,
 'MontezumaRevengeNoFrameskip-v4': 0.910187304019928,
 'PrivateEyeNoFrameskip-v4': 0.9924938678741456,
 'AsteroidsNoFrameskip-v4': 0.517248809337616,
 'BerzerkNoFrameskip-v4': 0.840388834476471,
 'BreakoutNoFrameskip-v4': 0.9140625,
 'DefenderNoFrameskip-v4': 0.4992050528526306,
 'RiverraidNoFrameskip-v4': 0.9315848350524902,
 'MsPacmanNoFrameskip-v4': 0.9165834784507751}

In [17]:
compute_overall_summary_dict(run_summary_dict)

{'small_object_localization': 0.3420865156740226,
 'agent_localization': 0.3454388289406672,
 'other_localization': 0.4941852057545991,
 'score_clock_lives_display': 0.5516124189688962,
 'level_room': 0.6836566946918509,
 'direction': 0.3821022727272727,
 'overall': nan}

In [11]:
methods = ["random-cnn", "vae", "cpc", "spatial-appo", "supervised"]
gbg_dict = {env:{} for env in atari_dict.keys()}

overall_dict = {method:{} for method in methods}

for method in methods:
    overall_summary_dict, run_summary_dict, _ = get_dicts_for(method)
    for env, dic in run_summary_dict.items():
        if len(dic) > 0:
            gbg_dict[env.lower()][method] = dic["overall"]
        else:
            gbg_dict[env.lower()][method] = "n/a"
    overall_dict[method] = overall_summary_dict["overall"]

print(" & ".join(["game"] + methods), "\\\\")

for game, dic in gbg_dict.items():
    if game in ["videopinball", "asteroids"]:
        continue
    print(" & ".join([game] + ["%8.2f"%(100*dic[k]) if k in dic else "n/a" for k in methods ]), "\\\\")


print(" & ".join(["overall"] + ["%8.2f"%(100*overall_dict[k]) if k in overall_dict else "n/a" for k in methods ]), "\\\\") 

In [7]:
overall_keys = ["overall"]
loc_keys = ['localization',
 'small_object_localization',
 'agent_localization',
 'enemy_localization']
misc_keys=['relative_position',
 'direction',
 'score',
 'level_room',
 'count_display',
 'existence',
 'speed']

In [10]:

print(" & ".join(loc_keys))
for method in methods:
    overall_summary_dict, run_summary_dict, _ = get_dicts_for(method)


    loc_nums = ["%8.2f"%(100*overall_summary_dict[k]) for k in loc_keys]
    print(method + " & "," & ".join(loc_nums), "\\\\")

print(" & ".join(misc_keys))    
for method in methods:
    overall_summary_dict, run_summary_dict, _ = get_dicts_for(method)



    misc_nums = ["%8.2f"%(100*overall_summary_dict[k]) for k in misc_keys]
    print(method + " & "," & ".join(misc_nums), "\\\\")

In [10]:
#rl agent parsing

In [9]:
loc_keys = ['overall',
 'localization',
 'small_object_localization',
 'agent_localization',
 'enemy_localization']

keys = ['overall',
 'small_object_localization',
 'agent_localization',
 'enemy_localization',"score_lives", 
 'count_display','level_room','direction', 'existence']

summary_stats = {env:{sk:0 for sk in summary_key_dict.keys()} for env in atari_dict.keys()}

for env in atari_dict.keys():
    for k in atari_dict[env].keys():
        for sk,v in summary_key_dict.items():
            if k in v:
                summary_stats[env][sk] +=1

print(" & ".join(["game"] + list(keys)))

for env,v in summary_stats.items():
    print( " & ".join([env] + list(str(v[k]) for k in keys)), "\\\\")



def get_pretrained_rl_dicts(algo):
    api = wandb.Api()
    runs = list(api.runs("curl-atari/curl-atari-2", {"config.method": "pretrained-rl-agent",
                                                     "config.zoo_algos":[algo],
                                                     "config.probe_collect_mode":"atari_zoo",
                                                     "state": "finished"}))

    runs = [run for run in runs if "test_mean_reward_per_episode" in run.summary_metrics.keys()]

    run_info_dict = {run.config["env_name"].replace("NoFrameskip-v4","")  :run.config for run in runs}
    run_dict = {run.config["env_name"].replace("NoFrameskip-v4","")  :run.summary_metrics for run in runs}

    run_summary_dict = {env_meth:{} for env_meth in run_dict.keys()}

    overall_summary_dict = {k:[] for k in list(summary_key_dict.keys()) + ["reward_per_episode"]}

    for run_name, run in run_dict.items():

        #print(run_name)
        for summary_name, key_list in summary_key_dict.items():
            run_mean_values = {k.lower():v for k,v in run.items() if "acc" in k and "stderr" not in k\
                               and  any(sum_key.lower() in k.lower() for sum_key in key_list)}
            #run_stderr_values = {k:v for k,v in run.items() if "stderr" in k and "mean" not in k and any(sum_key in k.lower() for sum_key in key_list)}
            if len(run_mean_values) > 0:
                run_summary = np.mean(list(run_mean_values.values()))
                overall_summary_dict[summary_name].append(run_summary)
                run_summary_dict[run_name][summary_name] = run_summary
        overall_summary_dict["reward_per_episode"].append(run["test_mean_reward_per_episode"])
        run_summary_dict[run_name]["reward_per_episode"] = run["test_mean_reward_per_episode"]
                #print("\t",summary_name,run_summary_dict[run_name][summary_name])
    overall_summary_dict = {k:np.mean(v) for k,v in overall_summary_dict.items()} 
    return overall_summary_dict, run_summary_dict #, run_dict, run_info_dict

rlag = {k:{} for k in atari_dict.keys()}

for algo in ["a2c", "apex"]:
    _, run_summary_dict = get_pretrained_rl_dicts(algo)
    for k, v in run_summary_dict.items():
        rlag[k.lower()][algo] = (v["overall"]*100,v['reward_per_episode']) 
    

print(" & ".join(["game", "a2c probe", "a2c returns","apex probe", "a2c returns"]), "\\\\")
for game, dic in rlag.items():
    #print(dic)
    if len(dic) > 1:
        all_res = list(dic["a2c"]) + list(dic["apex"])
        all_res = ["%.2f"%(float(n)) for n in all_res]
        print(game, " & " ," & ".join(all_res), "\\\\")
    


rlov = {}

for algo in ["a2c", "apex"]:
    osd, _ = get_pretrained_rl_dicts(algo)
    rlov[algo + "probe avg"] = osd["overall"]*100
    rlov[algo + "avg return per ep per game"] = osd['reward_per_episode']


a2c0,a2c1 = ["%8.2f"%rlov[k] for k in rlov.keys() if "a2c" in k]

apex0,apex1 = ["%8.2f"%rlov[k] for k in rlov.keys() if "apex" in k]

alla2c_probes = [v["a2c"][0] for v in rlag.values() if len(v) > 0]

alla2c_returns = [v["a2c"][1] for v in rlag.values() if len(v) > 0]

a2c_corr = np.corrcoef(alla2c_probes,alla2c_returns)[0][1]

allapex_probes =  [v["apex"][0] for v in rlag.values() if len(v) > 0]
allapex_returns =  [v["apex"][1] for v in rlag.values() if len(v) > 0]

apex_corr = np.corrcoef(allapex_probes,allapex_returns)[0][1]

print(" & ".join(["method", "probe_score", "avg_returns all games","correlation"]))
print(" & ".join(["a2c",a2c0,a2c1,str(a2c_corr)]))
print(" & ".join(["apex",apex0,apex1,str(apex_corr)]))